In [1]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_excel(r"C:\Users\VIKRAM\Downloads\NLP DATASET.xlsx")

In [3]:
df.head()

,type,meal type,person type,diet type,recommend
0,veg,breakfast,atheletic,high protein,"panner salad with some brown bread,add some oa..."
1,non veg,breakfast,normal,normal,idly or dosa or any type of breakfast
2,veg,breakfast,weight loss,low carb,"fruits and panner,sugar free foods"
3,non veg,breakfast,weight gain,high protein,eggs and chicken salad and some beetroot juice
4,non veg,lunch,atheletic,high protein,chicken breast and some vegetables like capsic...


In [4]:
df.tail()

,type,meal type,person type,diet type,recommend
15,non veg,lunch,weight gain,high protein,chicken curry or fry and rice and add some veg...
16,non veg,lunch,atheletic,high protein,chicken breast and some vegetables like capsic...
17,non veg,lunch,normal,normal,chicken curry or fry and rice and add some veg...
18,veg,dinner,weight loss,low carb,chapathi kurma milk
19,non veg,dinner,weight gain,high protein,"chicken breast,salmon,one egg"


In [5]:
df.isnull().sum()

type           0
meal type      0
person type    0
diet type      0
recommend      0
dtype: int64

In [6]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    
    words = word_tokenize(text)
    
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    return ' '.join(words)

In [7]:
df['type'] = df['type'].apply(preprocess_text)
df['meal type'] = df['meal type'].apply(preprocess_text)
df['person type'] = df['person type'].apply(preprocess_text)
df['diet type'] = df['diet type'].apply(preprocess_text)


df['features'] = df['type'] + " " + df['meal type'] + " " + df['person type'] + " " + df['diet type']

In [8]:
df['features']

0           veg breakfast atheletic high protein
1                non veg breakfast normal normal
2             veg breakfast weight loss low carb
3     non veg breakfast weight gain high protein
4           non veg lunch atheletic high protein
5                    non veg lunch normal normal
6             non veg lunch weight loss low carb
7            non veg lunch weight gain high carb
8              veg dinner atheletic high protein
9                   non veg dinner normal normal
10               veg dinner weight loss low carb
11       non veg dinner weight gain high protein
12      non veg breakfast atheletic high protein
13                   veg breakfast normal normal
14        non veg breakfast weight loss low carb
15        non veg lunch weight gain high protein
16          non veg lunch atheletic high protein
17                   non veg lunch normal normal
18               veg dinner weight loss low carb
19       non veg dinner weight gain high protein
Name: features, dtyp

In [9]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['features'])
tfidf_matrix

<20x14 sparse matrix of type '<class 'numpy.float64'>'
	with 114 stored elements in Compressed Sparse Row format>

In [10]:
def recommend_food_nlp(meal_type, food_type, person_type, diet_type):
    
    food_type = preprocess_text(food_type)
    meal_type = preprocess_text(meal_type)
    person_type = preprocess_text(person_type)
    diet_type = preprocess_text(diet_type)
    
    
    user_input = f"{food_type} {meal_type} {person_type} {diet_type}"
    
    
    user_tfidf = vectorizer.transform([user_input])
    
    
    similarities = cosine_similarity(user_tfidf, tfidf_matrix)
    
    
    idx = similarities.argsort()[0][-1]
    
    
    return df['recommend'].iloc[idx]


In [11]:
meal_type = input("Enter meal type (e.g., breakfast, lunch, dinner): ").strip()
food_type = input("Enter food type (e.g., veg, non-veg): ").strip()
person_type = input("Enter preferance type (e.g., athletic, normal, weight loss): ").strip()
diet_type = input("Enter diet type (e.g., high protein, low carb): ").strip()

recommendation = recommend_food_nlp(meal_type, food_type, person_type, diet_type)
print("Recommended food:", recommendation)


Recommended food:  chicken breast,salmon,one egg


interface

In [12]:
import gradio as gr

In [13]:
def recommend(meal_type, food_type, person_type, diet_type):
    recommendation = recommend_food_nlp(meal_type, food_type, person_type, diet_type)
    return f"Recommended food: {recommendation}"

In [15]:
interface = gr.Interface(
    fn=recommend, 
    inputs=[
        gr.Textbox(label="Meal Type (e.g., breakfast, lunch, dinner)"),
        gr.Textbox(label="Food Type (e.g., veg, non-veg)"),
        gr.Textbox(label="Preference Type (e.g., athletic, normal, weight loss)"),
        gr.Textbox(label="Diet Type (e.g., high protein, low carb)")
    ],
    outputs="text",  
    title="Food Recommendation System",
    description="Enter your preferences to get a food recommendation based on your meal type, food type, person type, and diet type.",
)

# Step 7: Launch the Gradio app
interface.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
